# Functional Closures and Decorators

Sometimes we may have "cross-cutting" concerns: functionality which is not related to the "core" functionality of a function:

- logging
- authentication
- locking
- entry of the function into a global registry

Python has a syntax that uses two features of the language (scoping and first-class functions) to enable such operations to be separated from the core function.

First, though, let's see if you can guess what the following code prints:

In [1]:
a = 5
def foo():
    a = 10
    b = 15
    print(f'1 => a={a}, b={b}')  # '1 => a={}, b={}'.format(a, b)   Py 3.5+
    def bar():
        a = 20
        print(f'2 => a={a}, b={b}')
    bar()
    print(f'3 => a={a}, b={b}')
foo()
print(f'4 => a={a}')
print(f'5 => b={b}')


1 => a=10, b=15
2 => a=20, b=15
3 => a=10, b=15
4 => a=5


NameError: name 'b' is not defined

##  Closures
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing (i.e. `nonlocal`)
  * G = global
  * B = builtin (e.g., len() function)
  
```python
a = 'global scope'

def outer_func():
    b = 'local to outer_func()'
    def inner_func():
        c = 'local to inner_func()'
        print(b, 'enclosing/nonlocal scope')
        print(a, 'global scope')
```
                
* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which "remembers" a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [3]:
def make_adder(x):
    
    def adder(y):
        return x + y # Python uses LEGB to find 'x'
    
    return adder

add39 = make_adder(39)
add39(10)

49

In [4]:
add12 = make_adder(12)
add12(10)

22

In [5]:
add39(10)

49

In [6]:
cell = add39.__closure__[0]

In [7]:
cell.cell_contents

39

In [8]:
x = make_adder(39)
x(10)

49

In [9]:
make_adder(39)(10)

49

In [10]:
add6 = make_adder(6)

In [11]:
print(add6(10), add39(10))

16 49


In [12]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
#type(add39)
repr(add39)

'<function make_adder.<locals>.adder at 0x111159bf8>'

In [13]:
repr(add6)

'<function make_adder.<locals>.adder at 0x111159f28>'

In [14]:
# all functions have a closure attribute
add39.__closure__

(<cell at 0x111101288: int object at 0x10f11b550>,)

In [15]:
# notice that the cell object has a reference to an int object
add39.__closure__[0].cell_contents

39

In [16]:
add6.__closure__[0].cell_contents

6

* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [17]:
def logging(function):
    print('Wrapping {}'.format(function))
    def wrapper(*args, **kwargs):
        print('Calling %r(%r, %r)' % (function, args, kwargs))
        return function(*args, **kwargs)
    return wrapper

logging_add39 = logging(add39)
print(logging_add39)
#print(add39(5)) # remember that add39 just adds 39 to our argument
#logging_add39(10)

Wrapping <function make_adder.<locals>.adder at 0x111159bf8>
<function logging.<locals>.wrapper at 0x111198268>


In [18]:
logging_add39.__closure__[0].cell_contents

<function __main__.make_adder.<locals>.adder(y)>

In [19]:
logging_add39(5)

Calling <function make_adder.<locals>.adder at 0x111159bf8>((5,), {})


44

In [20]:
add39 = logging(add39)

Wrapping <function make_adder.<locals>.adder at 0x111159bf8>


In [21]:
add39(5)

Calling <function make_adder.<locals>.adder at 0x111159bf8>((5,), {})


44

## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


In [22]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

In [23]:
def printme(me):
    print(me)

proxy_printme = proxy(printme)

In [24]:
proxy_printme('Hello there')

Hello there


## Decorators build on topics we already know...
* nested functions
* variable positional args (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [25]:
def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

def add_ints(a, b):
    return a + b

print('Running plain old add_ints()')
print(add_ints(3, 5))



Running plain old add_ints()
8


In [26]:
# manual decorator assignment
cooler_add_ints = document_it(add_ints) 

print('Running cooler add_ints()')
cooler_add_ints(3, 5)

Running cooler add_ints()
Running function: add_ints
Positional arguments: (3, 5)
Keyword arguments: {}
Result: 8


8

In [27]:
def add_ints(a, b):
    return a + b
add_ints = document_it(add_ints)

In [28]:
# decorator shorthand for what we did above
@document_it
def add_ints(a, b):
    return a + b

add_ints(4, b=7)

Running function: add_ints
Positional arguments: (4,)
Keyword arguments: {'b': 7}
Result: 11


11

Not all decorators wrap. Two examples:

In [29]:
# Flask-like routing

routes = {}  # map url => python function


def route(path):   # "decorator factory" (returns a decorator)
    print(f'Build route({path})')
    def decorator(function):   # decorator (takes a single function as input)
        print(f'Call decorator({function})')
        routes[path] = function
        # (no wrapping going on here)
        return function
    return decorator


@route('/')
def get_home():
    return 'Welcome'

@route('/about')
def get_about():
    return 'About us'

# @route('/error')
# def get_error():
#     return '404'

route_error_decorator = route('/error')
def get_error():
    return '404'
get_error = route_error_decorator(get_error)


Build route(/)
Call decorator(<function get_home at 0x1111987b8>)
Build route(/about)
Call decorator(<function get_about at 0x111198b70>)
Build route(/error)
Call decorator(<function get_error at 0x111198e18>)


In [30]:
routes

{'/': <function __main__.get_home()>,
 '/about': <function __main__.get_about()>,
 '/error': <function __main__.get_error()>}

In [31]:
def run_request(routes, url):
    function_to_call = routes[url]
    return function_to_call()

In [32]:
run_request(routes, '/')

'Welcome'

In [33]:
run_request(routes, '/about')

'About us'

In [34]:
run_request(routes, '/error')

'404'

In [35]:
# Cherrypy/Turbogears-like routing

def expose(function):
    function._exposed = True
    return function

class Controller(object):
    @expose
    def home(self):
        return 'Welcome'
    
c = Controller()

In [36]:
def dispatch_url(root_controller, url):
    parts = url.split('/')
    cur = root_controller
    for part in parts:
        next = getattr(cur, part)
        if not next._exposed:
            raise ValueError('403 Forbidden')
        cur = next
    return cur()

In [37]:
dispatch_url(c, 'home')

'Welcome'

```python
@EXPR0  # (
@EXPR1  # (
def FUNC():
    ...
# ))
```
    
... means nothing more than ...

```python
_temp0 = EXPR0
_temp1 = EXPR1
def FUNC():
    ...
FUNC = _temp1(FUNC)
FUNC = _temp0(FUNC)
```

# Lab

Open the [Closures Lab][closures-lab]

[closures-lab]: ./closures-lab.ipynb
